memperhatikan customer experience dengan menentukan pelanggan akan berhenti berlangganan (churn) atau tidak melalui model prediksi 

Langkah yang akan dilakukan
- Mencari ID pelanggan (Nomor telphone) yang valid
- Mengatasi data-data yang masih kosong (Missing Values)
- Mengatasi Nilai-Nilai Pencilan (Outlier) dari setiap Variable
- Menstandarisasi Nilai dari Variable

Library yang digunakan
- Pandas
    - library Python yang fokus untuk proses analisis data seperti manipulasi data, persiapan data, dan pembersihan data.
        - read_csv() digunakan untuk membaca file csv
        - str.match() digunakan untuk mencocokan dengan karakter tertentu
        - drop() digunakan untuk menghapus
        - count() digunakan untuk mengitung masing-masing variable
        - drop_duplicates() digunakan untuk menghapus data duplicate rows
        - fillna() digunakan untuk mengisi dengan nilai tertentu
        - quantile() digunakan untuk melihat quantile ke tertentu
        - mask() menganti nilai tertentu jika kondisi memenuhi
        - astype() merunubah tipe data
        - value_counts() digunakan untuk mengitung unik dari kolom
        - sort_values() digunakan untuk sort values
        - isnull() digunakan untuk mendeteksi missing values
        - dropna() digunakan untuk menghapus missing values
        - replace() digunakan untuk mengganti nilai
        - Matplotlib
- Matplotlib
    - library Python yang fokus pada visualisasi data seperti membuat plot grafik. Dapat digunakan dalam skrip Python, Python dan IPython shell, server aplikasi web, dan beberapa toolkit graphical user interface (GUI) lainnya.
        - figure() digunakan untuk membuat figure gambar baru
- Seaborn
    - Seaborn membangun di atas Matplotlib dan memperkenalkan tipe plot tambahan. Ini juga membuat plot Matplotlib tradisional lebih estetik.
        - box_plot() digunakan untuk membuat box plot

Data yang digunakan
```
df_load = pd.read_csv('https://dqlab-dataset.s3-ap-southeast-1.amazonaws.com/dqlab_telco.csv')
```
variabel yang dipake
- UpdatedAt Periode of Data taken
- customerID Customer ID
- gender Whether the customer is a male or a female (Male, Female)
- SeniorCitizen Whether the customer is a senior citizen or not (1, 0)
- Partner Whether the customer has a partner or not (Yes, No)
- Dependents Whether the customer has dependents or not (Yes, No)
- tenure Number of months the customer has stayed with the company
- PhoneService Whether the customer has a phone service or not (Yes, No)
- MultipleLines Whether the customer has multiple lines or not (Yes, No, No phone service)
- InternetService Customer’s internet service provider (DSL, Fiber optic, No)
- OnlineSecurity Whether the customer has online security or not (Yes, No, No internet service)
- OnlineBackup Whether the customer has online backup or not (Yes, No, No internet service)
- DeviceProtection Whether the customer has device protection or not (Yes, No, No internet service)
- TechSupport Whether the customer has tech support or not (Yes, No, No internet service)
- StreamingTV Whether the customer has streaming TV or not (Yes, No, No internet service)
- StreamingMovies Whether the customer has streaming movies or not (Yes, No, No internet service)
- Contract The contract term of the customer (Month-to-month, One year, Two year)
- PaperlessBilling Whether the customer has paperless billing or not (Yes, No)
- PaymentMethod The customer’s payment method (Electronic check, Mailed check, Bank transfer (automatic), Credit card (automatic))
- MonthlyCharges The amount charged to the customer monthly
- TotalCharges The total amount charged to the customer
- Churn Whether the customer churned or not (Yes or No)


In [7]:
# import library
import pandas as pd
# mempermudah penampilan row data
pd.options.display.max_columns = 50

# import dataset
df_load = pd.read_csv('https://dqlab-dataset.s3-ap-southeast-1.amazonaws.com/dqlab_telco.csv')

# Tampilkan jumlah baris dan kolom
print(f'{df_load.shape[0]} baris dan {df_load.shape[1]} kolom')

7113 baris dan 22 kolom


tampilkan 5 data teratas

In [4]:
df_load.head()

,UpdatedAt,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,202006,45759018157,Female,0,Yes,No,1.0,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,202006,45557574145,Male,0,No,No,34.0,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,202006,45366876421,Male,0,No,No,2.0,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,202006,45779536532,Male,0,No,No,45.0,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,202006,45923787906,Female,0,No,No,2.0,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


jumlah ID yang unik

In [9]:
df_load['customerID'].nunique()

7017

# Mencari Validitas ID Number Pelanggan

## Memfilter ID Number Pelanggan Format Tertentu
Mencari format ID Number (Phone Number) Pelanggan customerID yang benar
kriteria:
- Panjang karakter adalah 11-12
- Terdiri dari Angka Saja, tidak diperbolehkan ada karakter selain angka
- Diawali dengan angka 45 2 digit pertama

In [10]:
# astype() untuk merubah tipe datanya yang semula numeric
# str.match() & regex untuk mencocokan dengan kriteria diatas
df_load['valid_id'] = df_load['customerID'].astype(str).str.match(r'(45\d{9,10})')
df_load = (df_load[df_load['valid_id'] == True]).drop('valid_id', axis = 1)
# count() untuk menghitung banyaknya rows Customer ID
print('Hasil jumlah ID Customer yang terfilter adalah', df_load['customerID'].count())		

Hasil jumlah ID Customer yang terfilter adalah 7006


## Memfiltemr Duplikasi ID Number Pelanggan
tipe duplikasinya:
- Duplikasi dikarenakan inserting melebihi satu kali dengan nilai yang sama tiap kolomnya
- Duplikasi dikarenakan inserting beda periode pengambilan data

In [11]:
# Drop Duplicate Rows
df_load.drop_duplicates()
# Drop duplicate ID sorted by Periode
df_load = df_load.sort_values('UpdatedAt', ascending=False).drop_duplicates(['customerID'])
print('Hasil jumlah ID Customer yang sudah dihilangkan duplikasinya (distinct) adalah ', df_load['customerID'].count())		

Hasil jumlah ID Customer yang sudah dihilangkan duplikasinya (distinct) adalah  6993


# Mengatasi Missing Values

## Mengatasi Missing Values dengan Penghapusan Rows
menghapus Rows dari data-data yang tidak terdeteksi apakah dia churn atau tidak

In [12]:
# isnull() digunakan untuk mendeteksi missing values 
print('Total missing values data dari kolom Churn',df_load['Churn'].isnull().sum())
# dropna() untuk menghapus data dari kolom Churn yang missing values
df_load.dropna(subset=['Churn'],inplace=True)
print('Total Rows dan kolom Data setelah dihapus data Missing Values adalah',df_load.shape)

Total missing values data dari kolom Churn 43
Total Rows dan kolom Data setelah dihapus data Missing Values adalah (6950, 22)


## Mengatasi Missing Values dengan Pengisian Nilai tertentu

# Mengatasi Outlier

# Menstandarisasi Nilai